# Matchup Evaluator
This notebook is used to create code and logic that will help analyse an upcoming h2h Yahoo
Fantasy matchup

In [3]:
from src.data import all_functions as af
import pandas as pd

In [7]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

[2021-02-10 18:59:52,345 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-10 18:59:52,358 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2149.8055012226105
[2021-02-10 18:59:52,359 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [13]:
# Nunn 9Cat averages
nunn_of_yall_betta_team = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [14]:
nunn_of_yall_betta_team = nunn_of_yall_betta_team.loc["mean"]
nunn_of_yall_betta_team["PLAYER"] = af.NUNN_OF_YALL_BETTA["name"]
nunn_of_yall_betta_team = pd.DataFrame(nunn_of_yall_betta_team).T
nunn_of_yall_betta_team

PLAYER FG_PCT FT_PCT  FG3M    PTS   REB   AST   STL   BLK  \
mean  Nunn of Y'all Betta  0.457  0.817  1.92  15.21  4.96  3.70  1.00  0.39   

       TOV  
mean  1.63

In [15]:
# Crabbe 9Cat averages
magics_johnson_team = af.team_9cat_average_stats(af.MAGICS_JOHNSON, league)

[2021-02-08 19:15:55,461 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:15:55,464 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1616.0978128910065
[2021-02-08 19:15:55,464 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [16]:
magics_johnson_team = magics_johnson_team.loc["mean"]
magics_johnson_team["PLAYER"] = af.MAGICS_JOHNSON["name"]
magics_johnson_team = pd.DataFrame(magics_johnson_team).T
magics_johnson_team

PLAYER FG_PCT FT_PCT  FG3M    PTS   REB   AST   STL   BLK   TOV
mean  Magic's Johnson  0.453  0.801  2.19  16.77  5.93  3.88  1.03  0.53  1.72

In [17]:
# Drop PLAYER
nunn_of_yall_betta_team.drop("PLAYER", inplace=True, axis=1)
magics_johnson_team.drop("PLAYER", inplace=True, axis=1)

In [18]:
def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

In [19]:
nunn_of_yall_betta_team = columns_to_dtype_float(nunn_of_yall_betta_team)
magics_johnson_team = columns_to_dtype_float(magics_johnson_team)

In [23]:
# Difference
matchup_difference = nunn_of_yall_betta_team.sub(magics_johnson_team)
matchup_difference


FG_PCT  FT_PCT  FG3M   PTS   REB   AST   STL   BLK   TOV
mean   0.004   0.016 -0.27 -1.56 -0.97 -0.18 -0.03 -0.14 -0.09

In [24]:
matchup_difference.sum(axis=1)

mean   -3.22
dtype: float64

In [25]:
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def applymap_color_and_format(dataframe):
    """

    """
    fromated_matchup_difference = matchup_difference.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format(COLUMN_9CAT_DECIMAL_FORMAT)
    return fromated_matchup_difference

In [26]:
matchup_difference.style.applymap(
    af.color_negative_red, subset=pd.IndexSlice[:,["FG_PCT", "FT_PCT", "FG3M", "PTS", "REB", "AST",
                                                "STL", "BLK"]]).applymap(
    af.color_negative_red_tov, subset=pd.IndexSlice[:,["TOV"]]).format(COLUMN_9CAT_DECIMAL_FORMAT)

In [68]:
PLAYER_COLUMN = "PLAYER"
MEAN_ROW = "mean"
NAME_KEY = "name"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

def get_and_format_team_9cat(team_dict, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team_dict, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team_dict[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns differneces and returns a color coded
    and decimal place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = dataframe.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """
    Function authenticates on Yahoo and creates creates two dataframes containing team roster
    9cat averages for each team specified in the parameters. It calculates the difference between
    both 9cat averages and finally it formats the values for visual ease.
    params team1 (dictionary): team dictionary
    params team2 (dictionary): team dictionary
    return matchup_difference (dataframe): team difference 9cat dataframe
    return colored_matchup_difference (dataframe): formatted team difference 9cat dataframe
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

In [21]:
# Get difference dataframes
matchup_difference, colored_matchup_difference = compare_h2h_team_9cat(af.NUNN_OF_YALL_BETTA, af
                                                                       .CRABBEHERBYTHEPUSSY)

[2021-02-07 17:25:28,697 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 17:25:28,700 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 238.53321433067322
[2021-02-07 17:25:28,700 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [22]:
# Difference dataframe
matchup_difference

FG_PCT  FT_PCT  FG3M   PTS   REB  AST   STL   BLK  TOV
mean  -0.028  -0.003  0.42  1.04 -1.26  0.7  0.14 -0.52  0.0

In [23]:
# Formatted difference dataframe
colored_matchup_difference

In [24]:
# Wins/Losses
positive_list = list()
negative_list = list()

for value in matchup_difference.values.tolist()[0]:
    if value > 0:
        positive_list.append(value)
    elif value < 0:
        negative_list.append(abs(value))

In [25]:
# Print
print(f"Sum of positive list is: {sum(positive_list)}")
print(f"Sum of negative list is: {sum(negative_list)}")

Sum of positive list is: 2.299999999999999
Sum of negative list is: 1.8110000000000006


## Free Agent Sort by Rank
This section will work on logic that returns the Leaugue's FA sorted by Rank. This needs to be
done because it will be faster to calculate stats for the top 25 FA rather than all +400 in Yahoo.

In [36]:
from yahoo_fantasy_api import yhandler
import yahoo_fantasy_api as yfa
yh = yhandler.YHandler(sc)
gm = yfa.Game(sc, 'nba')
# Get League Ids I have access to
league_id_list = gm.league_ids(year=2020)
league_id = "".join(str(id) for id in league_id_list)

In [37]:
yh.get(f"/league/{league_id}/players;status=FA;sort=AR")

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/league/402.l.55374/players;status=FA;sort=AR',
  'league': [{'league_key': '402.l.55374',
    'league_id': '55374',
    'name': 'Lebrontourage',
    'url': 'https://basketball.fantasysports.yahoo.com/nba/55374',
    'logo_url': 'https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/cab87bf0fa8d01c5ba29f76654390cf6fed3776b77431028f7a94f67e6d67976.jpg',
    'draft_status': 'postdraft',
    'num_teams': 12,
    'edit_key': '2021-02-08',
    'weekly_deadline': 'intraday',
    'league_update_timestamp': '1612770592',
    'scoring_type': 'headone',
    'league_type': 'private',
    'renew': '395_53528',
    'renewed': '',
    'iris_group_chat_id': '',
    'allow_add_to_dl_extra_pos': 0,
    'is_pro_league': '0',
    'is_cash_league': '0',
    'current_week': 8,
    'start_week': '1',
    'start_date': '2020-12-22',
    'end_week': '20',
    'end_date': '2021-05-16',
    'game_code': 'nba',
    '

## Get Week Date
This section will create logic needed to return the current week number (i.e. Week 5). This will
allow for the date, start and end, to be found.


In [38]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)


[2021-02-08 19:23:28,626 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:23:28,629 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2069.263598918915
[2021-02-08 19:23:28,631 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [39]:
# Current week
current_fantasy_week = league.current_week()

# Next week
next_fantasy_week = current_fantasy_week + 1
print(f"Next weeks fantasy mathcup will fall under week {next_fantasy_week}")

Next weeks fantasy mathcup will fall under week 9


In [40]:
# Get dates
week_start_date = league.week_date_range(current_fantasy_week)[0]

In [41]:
week_start_date.strftime("%A, %d %B %Y")

'Monday, 08 February 2021'

## NBA Schedule
This section creates logic to return the NBA schedule. The returned JSON will be filtered by
start and end date in order to return games scheduled in a desired Yahoo Fantasy week.

In [42]:
import requests
r = requests.get("http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2020/league/00_full_schedule.json")

In [43]:
r.json()['lscd']

[{'mscd': {'mon': 'December',
   'g': [{'gid': '0012000001',
     'gcode': '20201211/ORLATL',
     'seri': '',
     'is': 1,
     'gdte': '2020-12-11',
     'htm': '2020-12-11T19:00:00',
     'vtm': '2020-12-11T19:00:00',
     'etm': '2020-12-11T19:00:00',
     'an': 'State Farm Arena',
     'ac': 'Atlanta',
     'as': 'GA',
     'st': '3',
     'stt': 'Final',
     'bd': {'b': [{'seq': 1,
        'disp': 'Fox Sports Southeast - Atlanta',
        'scope': 'home',
        'type': 'tv',
        'lan': 'English'},
       {'seq': 2,
        'disp': 'TSN',
        'scope': 'can',
        'type': 'tv',
        'lan': 'English'},
       {'seq': 3,
        'disp': 'WZGC 92.9 FM The Game',
        'scope': 'home',
        'type': 'radio',
        'lan': 'English'},
       {'seq': 4,
        'disp': 'WYGM 96.9FM / 740AM',
        'scope': 'away',
        'type': 'radio',
        'lan': 'English'}]},
     'v': {'tid': 1610612753,
      're': '1-0',
      'ta': 'ORL',
      'tn': 'Magic',
      't

In [44]:
import requests

LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe


In [45]:
season_schedule_2020 = get_game_information_in_season("2020")
season_schedule_2020

gdte         gid                    home      home_tid  \
0   2020-12-11  0012000001           Atlanta Hawks  1.610613e+09   
1   2020-12-11  0012000002         Detroit Pistons  1.610613e+09   
2   2020-12-11  0012000003           Chicago Bulls  1.610613e+09   
3   2020-12-11  0012000004      Los Angeles Lakers  1.610613e+09   
4   2020-12-11  0012000005  Portland Trail Blazers  1.610613e+09   
..         ...         ...                     ...           ...   
26  2021-03-04  0022000555       Memphis Grizzlies  1.610613e+09   
27  2021-03-04  0022000556       San Antonio Spurs  1.610613e+09   
28  2021-03-04  0022000557    New Orleans Pelicans  1.610613e+09   
29  2021-03-04  0022000185            Phoenix Suns  1.610613e+09   
30  2021-03-04  0022000558  Portland Trail Blazers  1.610613e+09   

                 visitors  visitors_tid  
0           Orlando Magic  1.610613e+09  
1         New York Knicks  1.610613e+09  
2         Houston Rockets  1.610613e+09  
3             LA Clippers  1.610613e+09  
4        Sacramento Kings  1.610613e+09  
..                    ...           ...  
26        Milwaukee Bucks  1.610613e+09  
27  Oklahoma City Thunder  1.610613e+09  
28             Miami Heat  1.610613e+09  
29  Golden State Warriors  1.610613e+09  
30       Sacramento Kings  1.610613e+09  

[611 rows x 6 columns]

## Game Stats in Week
This section will combine the information returned when querying for season schedule with the
logic used to find start/end dates in a Yahoo Fantasy week. The goal is to return the count of
games a specific player plays in a given Yahoo Fantasy week

In [46]:
# Current week
week_start_date = league.week_date_range(current_fantasy_week)[0]
week_end_date = league.week_date_range(current_fantasy_week)[1]
print(f"The current week start date is: {week_start_date}")
print(f"The current week end date is: {week_end_date}")

The current week start date is: 2021-02-08
The current week end date is: 2021-02-14


In [47]:
season_schedule_2020["gdte"] = pd.to_datetime(season_schedule_2020["gdte"])
season_schedule_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611 entries, 0 to 30
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gdte          611 non-null    datetime64[ns]
 1   gid           611 non-null    object        
 2   home          611 non-null    object        
 3   home_tid      611 non-null    float64       
 4   visitors      611 non-null    object        
 5   visitors_tid  611 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 33.4+ KB


In [48]:
# Current week games
current_fantasy_games_week = season_schedule_2020[(season_schedule_2020["gdte"] >= \
                                                   str(week_start_date)) &
                                                  (season_schedule_2020["gdte"] <= \
                                                   str(week_end_date))]
current_fantasy_games_week

gdte         gid                    home      home_tid  \
57  2021-02-08  0022000367       Charlotte Hornets  1.610613e+09   
58  2021-02-08  0022000368           Chicago Bulls  1.610613e+09   
59  2021-02-08  0022000369       Memphis Grizzlies  1.610613e+09   
60  2021-02-08  0022000370        Dallas Mavericks  1.610613e+09   
61  2021-02-08  0022000371       San Antonio Spurs  1.610613e+09   
62  2021-02-08  0022000372            Phoenix Suns  1.610613e+09   
63  2021-02-08  0022000373          Denver Nuggets  1.610613e+09   
64  2021-02-08  0022000374      Los Angeles Lakers  1.610613e+09   
65  2021-02-09  0022000375         Detroit Pistons  1.610613e+09   
66  2021-02-09  0022000376              Miami Heat  1.610613e+09   
67  2021-02-09  0022000377    New Orleans Pelicans  1.610613e+09   
68  2021-02-09  0022000378       San Antonio Spurs  1.610613e+09   
69  2021-02-09  0022000379               Utah Jazz  1.610613e+09   
70  2021-02-09  0022000380  Portland Trail Blazers  1.610613e+09   
71  2021-02-09  0022000381        Sacramento Kings  1.610613e+09   
72  2021-02-10  0022000382      Washington Wizards  1.610613e+09   
73  2021-02-10  0022000384        Dallas Mavericks  1.610613e+09   
74  2021-02-10  0022000383           Brooklyn Nets  1.610613e+09   
75  2021-02-10  0022000386       Memphis Grizzlies  1.610613e+09   
76  2021-02-10  0022000387  Minnesota Timberwolves  1.610613e+09   
77  2021-02-10  0022000385           Chicago Bulls  1.610613e+09   
78  2021-02-10  0022000388          Denver Nuggets  1.610613e+09   
79  2021-02-10  0022000389            Phoenix Suns  1.610613e+09   
80  2021-02-10  0022000390      Los Angeles Lakers  1.610613e+09   
81  2021-02-11  0022000392         Houston Rockets  1.610613e+09   
82  2021-02-11  0022000398          Boston Celtics  1.610613e+09   
83  2021-02-11  0022000391         Detroit Pistons  1.610613e+09   
84  2021-02-11  0022000393   Golden State Warriors  1.610613e+09   
85  2021-02-11  0022000394  Portland Trail Blazers  1.610613e+09   
86  2021-02-12  0022000395       Charlotte Hornets  1.610613e+09   
87  2021-02-12  0022000396      Washington Wizards  1.610613e+09   
88  2021-02-12  0022000397           Atlanta Hawks  1.610613e+09   
89  2021-02-12  0022000399        Dallas Mavericks  1.610613e+09   
90  2021-02-12  0022000400           Chicago Bulls  1.610613e+09   
91  2021-02-12  0022000412          Boston Celtics  1.610613e+09   
92  2021-02-12  0022000401          Denver Nuggets  1.610613e+09   
93  2021-02-12  0022000402               Utah Jazz  1.610613e+09   
94  2021-02-12  0022000403      Los Angeles Lakers  1.610613e+09   
95  2021-02-12  0022000404  Portland Trail Blazers  1.610613e+09   
96  2021-02-12  0022000405        Sacramento Kings  1.610613e+09   
97  2021-02-13  0022000406            Phoenix Suns  1.610613e+09   
98  2021-02-13  0022000407           Atlanta Hawks  1.610613e+09   
99  2021-02-13  0022000408         New York Knicks  1.610613e+09   
100 2021-02-13  0022000409   Golden State Warriors  1.610613e+09   
101 2021-02-13  0022000410               Utah Jazz  1.610613e+09   
102 2021-02-14  0022000560      Washington Wizards  1.610613e+09   
103 2021-02-14  0022000413         Toronto Raptors  1.610613e+09   
104 2021-02-14  0022000414       Charlotte Hornets  1.610613e+09   
105 2021-02-14  0022000415   Oklahoma City Thunder  1.610613e+09   
106 2021-02-14  0022000561         Detroit Pistons  1.610613e+09   
107 2021-02-14  0022000411        Dallas Mavericks  1.610613e+09   
108 2021-02-14  0022000417            Phoenix Suns  1.610613e+09   
109 2021-02-14  0022000418        Sacramento Kings  1.610613e+09   
110 2021-02-14  0022000416          Denver Nuggets  1.610613e+09   
111 2021-02-14  0022000419             LA Clippers  1.610613e+09   

                   visitors  visitors_tid  
57          Houston Rockets  1.610613e+09  
58       Washington Wizards  1.610613e+09  
59          Toronto Raptors  1.610613e+09  
60   Minnesota Timberwolv

In [49]:
current_fantasy_games_week[["home", "visitors"]].value_counts()

home                    visitors              
San Antonio Spurs       Golden State Warriors     2
Los Angeles Lakers      Oklahoma City Thunder     2
Atlanta Hawks           Indiana Pacers            1
Portland Trail Blazers  Cleveland Cavaliers       1
Memphis Grizzlies       Toronto Raptors           1
Miami Heat              New York Knicks           1
Minnesota Timberwolves  LA Clippers               1
New Orleans Pelicans    Houston Rockets           1
New York Knicks         Houston Rockets           1
Oklahoma City Thunder   Milwaukee Bucks           1
Phoenix Suns            Cleveland Cavaliers       1
                        Milwaukee Bucks           1
                        Orlando Magic             1
                        Philadelphia 76ers        1
Portland Trail Blazers  Orlando Magic             1
                        Philadelphia 76ers        1
Sacramento Kings        Memphis Grizzlies         1
                        Orlando Magic             1
                 

In [50]:
current_fantasy_games_week["visitors"].value_counts()

Orlando Magic             4
Milwaukee Bucks           4
Cleveland Cavaliers       4
Philadelphia 76ers        3
Houston Rockets           3
Oklahoma City Thunder     3
New Orleans Pelicans      3
Minnesota Timberwolves    3
Toronto Raptors           3
Indiana Pacers            3
Brooklyn Nets             2
New York Knicks           2
Memphis Grizzlies         2
Miami Heat                2
LA Clippers               2
Golden State Warriors     2
Boston Celtics            2
San Antonio Spurs         2
Washington Wizards        1
Portland Trail Blazers    1
Detroit Pistons           1
Los Angeles Lakers        1
Charlotte Hornets         1
Atlanta Hawks             1
Name: visitors, dtype: int64

In [51]:
def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """

    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >= \
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <= \
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace("LA Clippers", "Los Angeles Clippers")
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        player_games_dictionary[player] = \
            team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
    return player_games_dictionary

In [52]:
tm = league.to_team(af.NUNN_OF_YALL_BETTA["team_key"])
team_roster = tm.roster()
player_id_team_list = [{"playerid" : player["player_id"], "playername" : player["name"]} for player in
                       team_roster]
player_id_team_list

[{'playerid': 6426, 'playername': 'Theo Maledon'},
 {'playerid': 5667, 'playername': 'Malcolm Brogdon'},
 {'playerid': 4660, 'playername': 'Patty Mills'},
 {'playerid': 5073, 'playername': 'Khris Middleton'},
 {'playerid': 4912, 'playername': 'Jimmy Butler'},
 {'playerid': 5352, 'playername': 'Nikola Jokic'},
 {'playerid': 4926, 'playername': 'Davis Bertans'},
 {'playerid': 5658, 'playername': 'Pascal Siakam'},
 {'playerid': 4749, 'playername': 'Eric Bledsoe'},
 {'playerid': 6173, 'playername': 'Cameron Johnson'},
 {'playerid': 5317, 'playername': 'Marcus Smart'},
 {'playerid': 5018, 'playername': 'Jeremy Lamb'},
 {'playerid': 6073, 'playername': 'Duncan Robinson'},
 {'playerid': 4899, 'playername': 'Enes Kanter'},
 {'playerid': 6048, 'playername': 'Gary Trent Jr.'}]

In [53]:
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"

def get_player_ids_names_in_fantasy_team(team_dict):
    """

    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

In [54]:
# Get ids/names in Fantasy team
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                                 .NUNN_OF_YALL_BETTA)

[2021-02-08 19:23:39,344 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:23:39,356 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2079.9905524253845
[2021-02-08 19:23:39,357 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [55]:
player_name_list

['Theo Maledon',
 'Malcolm Brogdon',
 'Patty Mills',
 'Khris Middleton',
 'Jimmy Butler',
 'Nikola Jokic',
 'Davis Bertans',
 'Pascal Siakam',
 'Eric Bledsoe',
 'Cameron Johnson',
 'Marcus Smart',
 'Jeremy Lamb',
 'Duncan Robinson',
 'Enes Kanter',
 'Gary Trent Jr.']

In [56]:
af.yahoo_player_team_and_jersey("Marcus Smart")[0]


current_fantasy_games_week

[2021-02-08 19:23:43,763 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:23:43,768 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2084.402400493622
[2021-02-08 19:23:43,769 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


gdte         gid                    home      home_tid  \
57  2021-02-08  0022000367       Charlotte Hornets  1.610613e+09   
58  2021-02-08  0022000368           Chicago Bulls  1.610613e+09   
59  2021-02-08  0022000369       Memphis Grizzlies  1.610613e+09   
60  2021-02-08  0022000370        Dallas Mavericks  1.610613e+09   
61  2021-02-08  0022000371       San Antonio Spurs  1.610613e+09   
62  2021-02-08  0022000372            Phoenix Suns  1.610613e+09   
63  2021-02-08  0022000373          Denver Nuggets  1.610613e+09   
64  2021-02-08  0022000374      Los Angeles Lakers  1.610613e+09   
65  2021-02-09  0022000375         Detroit Pistons  1.610613e+09   
66  2021-02-09  0022000376              Miami Heat  1.610613e+09   
67  2021-02-09  0022000377    New Orleans Pelicans  1.610613e+09   
68  2021-02-09  0022000378       San Antonio Spurs  1.610613e+09   
69  2021-02-09  0022000379               Utah Jazz  1.610613e+09   
70  2021-02-09  0022000380  Portland Trail Blazers  1.610613e+09   
71  2021-02-09  0022000381        Sacramento Kings  1.610613e+09   
72  2021-02-10  0022000382      Washington Wizards  1.610613e+09   
73  2021-02-10  0022000384        Dallas Mavericks  1.610613e+09   
74  2021-02-10  0022000383           Brooklyn Nets  1.610613e+09   
75  2021-02-10  0022000386       Memphis Grizzlies  1.610613e+09   
76  2021-02-10  0022000387  Minnesota Timberwolves  1.610613e+09   
77  2021-02-10  0022000385           Chicago Bulls  1.610613e+09   
78  2021-02-10  0022000388          Denver Nuggets  1.610613e+09   
79  2021-02-10  0022000389            Phoenix Suns  1.610613e+09   
80  2021-02-10  0022000390      Los Angeles Lakers  1.610613e+09   
81  2021-02-11  0022000392         Houston Rockets  1.610613e+09   
82  2021-02-11  0022000398          Boston Celtics  1.610613e+09   
83  2021-02-11  0022000391         Detroit Pistons  1.610613e+09   
84  2021-02-11  0022000393   Golden State Warriors  1.610613e+09   
85  2021-02-11  0022000394  Portland Trail Blazers  1.610613e+09   
86  2021-02-12  0022000395       Charlotte Hornets  1.610613e+09   
87  2021-02-12  0022000396      Washington Wizards  1.610613e+09   
88  2021-02-12  0022000397           Atlanta Hawks  1.610613e+09   
89  2021-02-12  0022000399        Dallas Mavericks  1.610613e+09   
90  2021-02-12  0022000400           Chicago Bulls  1.610613e+09   
91  2021-02-12  0022000412          Boston Celtics  1.610613e+09   
92  2021-02-12  0022000401          Denver Nuggets  1.610613e+09   
93  2021-02-12  0022000402               Utah Jazz  1.610613e+09   
94  2021-02-12  0022000403      Los Angeles Lakers  1.610613e+09   
95  2021-02-12  0022000404  Portland Trail Blazers  1.610613e+09   
96  2021-02-12  0022000405        Sacramento Kings  1.610613e+09   
97  2021-02-13  0022000406            Phoenix Suns  1.610613e+09   
98  2021-02-13  0022000407           Atlanta Hawks  1.610613e+09   
99  2021-02-13  0022000408         New York Knicks  1.610613e+09   
100 2021-02-13  0022000409   Golden State Warriors  1.610613e+09   
101 2021-02-13  0022000410               Utah Jazz  1.610613e+09   
102 2021-02-14  0022000560      Washington Wizards  1.610613e+09   
103 2021-02-14  0022000413         Toronto Raptors  1.610613e+09   
104 2021-02-14  0022000414       Charlotte Hornets  1.610613e+09   
105 2021-02-14  0022000415   Oklahoma City Thunder  1.610613e+09   
106 2021-02-14  0022000561         Detroit Pistons  1.610613e+09   
107 2021-02-14  0022000411        Dallas Mavericks  1.610613e+09   
108 2021-02-14  0022000417            Phoenix Suns  1.610613e+09   
109 2021-02-14  0022000418        Sacramento Kings  1.610613e+09   
110 2021-02-14  0022000416          Denver Nuggets  1.610613e+09   
111 2021-02-14  0022000419             LA Clippers  1.610613e+09   

                   visitors  visitors_tid  
57          Houston Rockets  1.610613e+09  
58       Washington Wizards  1.610613e+09  
59          Toronto Raptors  1.610613e+09  
60   Minnesota Timberwolv

In [84]:
teams_playing_home_visitors = current_fantasy_games_week["home"].append(current_fantasy_games_week["visitors"]).value_counts()

In [85]:
teams_playing_home_visitors = pd.DataFrame(teams_playing_home_visitors)
teams_playing_home_visitors.reset_index(inplace=True)
teams_playing_home_visitors

index  0
0          Milwaukee Bucks  4
1         Dallas Mavericks  4
2        Charlotte Hornets  4
3    Golden State Warriors  4
4          Houston Rockets  4
5      Cleveland Cavaliers  4
6           Denver Nuggets  4
7          Detroit Pistons  4
8            Orlando Magic  4
9          Toronto Raptors  4
10       San Antonio Spurs  4
11    New Orleans Pelicans  4
12      Washington Wizards  4
13            Phoenix Suns  4
14  Portland Trail Blazers  4
15       Memphis Grizzlies  4
16  Minnesota Timberwolves  4
17          Boston Celtics  4
18      Los Angeles Lakers  4
19   Oklahoma City Thunder  4
20        Sacramento Kings  3
21             LA Clippers  3
22              Miami Heat  3
23               Utah Jazz  3
24          Indiana Pacers  3
25           Brooklyn Nets  3
26         New York Knicks  3
27           Atlanta Hawks  3
28           Chicago Bulls  3
29      Philadelphia 76ers  3

In [86]:
teams_playing_home_visitors = teams_playing_home_visitors.replace("LA Clippers", "Los Angeles Clippers")
teams_playing_home_visitors

index  0
0          Milwaukee Bucks  4
1         Dallas Mavericks  4
2        Charlotte Hornets  4
3    Golden State Warriors  4
4          Houston Rockets  4
5      Cleveland Cavaliers  4
6           Denver Nuggets  4
7          Detroit Pistons  4
8            Orlando Magic  4
9          Toronto Raptors  4
10       San Antonio Spurs  4
11    New Orleans Pelicans  4
12      Washington Wizards  4
13            Phoenix Suns  4
14  Portland Trail Blazers  4
15       Memphis Grizzlies  4
16  Minnesota Timberwolves  4
17          Boston Celtics  4
18      Los Angeles Lakers  4
19   Oklahoma City Thunder  4
20        Sacramento Kings  3
21    Los Angeles Clippers  3
22              Miami Heat  3
23               Utah Jazz  3
24          Indiana Pacers  3
25           Brooklyn Nets  3
26         New York Knicks  3
27           Atlanta Hawks  3
28           Chicago Bulls  3
29      Philadelphia 76ers  3

In [91]:
teams_playing_home_visitors[teams_playing_home_visitors["index"] == "Los Angeles Clippers"][0].iloc[0]

3

In [71]:
teams_playing_home_visitors.replace(to_replace="LA Clippers", value="Los Angeles Clippers",
                                    regex=True)

Milwaukee Bucks           4
Dallas Mavericks          4
Charlotte Hornets         4
Golden State Warriors     4
Houston Rockets           4
Cleveland Cavaliers       4
Denver Nuggets            4
Detroit Pistons           4
Orlando Magic             4
Toronto Raptors           4
San Antonio Spurs         4
New Orleans Pelicans      4
Washington Wizards        4
Phoenix Suns              4
Portland Trail Blazers    4
Memphis Grizzlies         4
Minnesota Timberwolves    4
Boston Celtics            4
Los Angeles Lakers        4
Oklahoma City Thunder     4
Sacramento Kings          3
LA Clippers               3
Miami Heat                3
Utah Jazz                 3
Indiana Pacers            3
Brooklyn Nets             3
New York Knicks           3
Atlanta Hawks             3
Chicago Bulls             3
Philadelphia 76ers        3
dtype: int64

In [55]:
player_games_dictionary = dict()
for player in player_name_list:
    team = af.yahoo_player_team_and_jersey(player)[0]
    player_games_dictionary[player] = teams_playing_home_visitors[team]

[2021-02-07 17:29:13,963 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 17:29:13,967 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 463.8003680706024
[2021-02-07 17:29:13,967 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-07 17:29:14,925 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 17:29:14,928 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 464.76193165779114
[2021-02-07 17:29:14,929 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-07 17:29:15,839 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 17:29:15,842 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 465.6757547855377
[2021-02-07 17:29:15,843 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-07 17:29:16,752 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 17:29:16,755 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 466.58912205696106
[2021-02-07 17:29:16,756 DEBUG] [yahoo_oauth.oauth

In [56]:
player_games_dictionary

{'Marcus Smart': 4,
 'Malcolm Brogdon': 4,
 'Jeremy Lamb': 4,
 'Duncan Robinson': 4,
 'Jimmy Butler': 4,
 'Nikola Jokic': 3,
 'Enes Kanter': 5,
 'Davis Bertans': 4,
 'Eric Bledsoe': 4,
 'Cameron Johnson': 4,
 'Theo Maledon': 4,
 'Patty Mills': 3,
 'Gary Trent Jr.': 5,
 'Khris Middleton': 4,
 'Pascal Siakam': 3}

In [57]:
# Nunn 9Cat averages
nunn_of_yall_betta_team = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [58]:
a = nunn_of_yall_betta_team.copy()
a

PLAYER FG_PCT FT_PCT  FG3M    PTS    REB   AST   STL   BLK  \
0        Marcus Smart  0.394  0.742  1.65  13.06   2.76  6.06  1.82  0.53   
1     Malcolm Brogdon  0.452  0.889  2.74  21.96   4.00  6.65  1.35  0.22   
2         Jeremy Lamb  0.506  0.963  2.00  13.20   4.60  1.60  1.00  0.80   
3     Duncan Robinson  0.427  0.861  3.36  13.05   3.68  1.68  0.41  0.32   
4        Jimmy Butler  0.458  0.851  0.00  18.30   6.50  6.30  2.10  0.50   
5        Nikola Jokic  0.569  0.840  1.41  27.18  11.45  8.59  1.68  0.68   
6         Enes Kanter  0.596  0.765  0.00  11.41  10.91  1.09  0.59  0.91   
7       Davis Bertans  0.343  0.806  2.56  10.50   2.81  1.06  0.56  0.31   
8        Eric Bledsoe  0.444  0.651  2.29  13.71   3.67  3.71  0.67  0.29   
9     Cameron Johnson  0.443  0.870  2.24  11.29   3.67  1.57  0.67  0.33   
10       Theo Maledon  0.397  0.632  1.43   7.52   2.62  2.86  0.95  0.05   
11        Patty Mills  0.453  0.938  2.57  13.61   1.91  2.70  0.65  0.04   
12     Gary Trent Jr.  0.421  0.708  3.05  14.00   2.00  1.15  0.65  0.20   
13    Khris Middleton  0.520  0.928  2.61  20.43   6.00  5.78  0.96  0.13   
14      Pascal Siakam  0.454  0.802  0.90  19.65   7.95  4.55  1.15  0.60   
mean              NaN  0.458  0.816  1.92  15.26   4.97  3.69  1.01  0.39   

       TOV  
0     1.76  
1     1.87  
2     1.00  
3     1.59  
4     1.60  
5     3.77  
6     1.27  
7     0.75  
8     1.95  
9     0.81  
10    1.81  
11    0.87  
12    0.60  
13    2.26  
14    2.40  
mean  1.62

In [61]:
a[a["PLAYER"] == "Marcus Smart"].iloc[:, 3:] * 3

FG3M              PTS           REB           AST           STL  \
0  1.651.651.65  13.0613.0613.06  2.762.762.76  6.066.066.06  1.821.821.82   

            BLK           TOV  
0  0.530.530.53  1.761.761.76

In [62]:
pd.concat([a[a["PLAYER"] == "Marcus Smart"].iloc[:, :3], (a[a["PLAYER"] == "Marcus Smart"].iloc[:, 3:] * 3)], axis=1)

PLAYER FG_PCT FT_PCT          FG3M              PTS           REB  \
0  Marcus Smart  0.394  0.742  1.651.651.65  13.0613.0613.06  2.762.762.76   

            AST           STL           BLK           TOV  
0  6.066.066.06  1.821.821.82  0.530.530.53  1.761.761.76

In [69]:
PLAYER_COLUMN = "PLAYER"
def get_predicted_player_weekly_9cat(player_games_dictionary, team_9cat_average_stats):
    """
    Function is used to create a dataframe which contains predicted totals for a week of Yahoo
    Fantasy play. For each player, 9cat features are multiplied by the amount of games that
    player plays in one week.
    param player_games_dictionary (dictionary): key/value pairing of player names/games playing
    param team_9cat_average_stats (pandas dataframe): dataframe containing the season average
    9cat stats for each player on a team
    return predicted_9cat_stats (pandas dataframe): dataframe with 9cat averages multiplied by
    games played in a week
    """
    predicted_9cat_stats = pd.DataFrame()
    team_9cat_average_stats = columns_to_dtype_float(team_9cat_average_stats)
    for player in player_games_dictionary.keys():
        player_predicted_9cat_stats = pd.concat(
            [team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, :3],
            (team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, 3:]
            * 3)], axis=1)
        predicted_9cat_stats = player_predicted_9cat_stats.append(predicted_9cat_stats)
    predicted_9cat_stats.sort_index(inplace=True)
    return predicted_9cat_stats

In [70]:
predicted_weekly_df = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                    nunn_of_yall_betta_team)
predicted_weekly_df

PLAYER  FG_PCT  FT_PCT   FG3M    PTS    REB    AST   STL   BLK  \
0      Marcus Smart   0.394   0.742   4.95  39.18   8.28  18.18  5.46  1.59   
1   Malcolm Brogdon   0.452   0.889   8.22  65.88  12.00  19.95  4.05  0.66   
2       Jeremy Lamb   0.506   0.963   6.00  39.60  13.80   4.80  3.00  2.40   
3   Duncan Robinson   0.427   0.861  10.08  39.15  11.04   5.04  1.23  0.96   
4      Jimmy Butler   0.458   0.851   0.00  54.90  19.50  18.90  6.30  1.50   
5      Nikola Jokic   0.569   0.840   4.23  81.54  34.35  25.77  5.04  2.04   
6       Enes Kanter   0.596   0.765   0.00  34.23  32.73   3.27  1.77  2.73   
7     Davis Bertans   0.343   0.806   7.68  31.50   8.43   3.18  1.68  0.93   
8      Eric Bledsoe   0.444   0.651   6.87  41.13  11.01  11.13  2.01  0.87   
9   Cameron Johnson   0.443   0.870   6.72  33.87  11.01   4.71  2.01  0.99   
10     Theo Maledon   0.397   0.632   4.29  22.56   7.86   8.58  2.85  0.15   
11      Patty Mills   0.453   0.938   7.71  40.83   5.73   8.10  1.95  0.12   
12   Gary Trent Jr.   0.421   0.708   9.15  42.00   6.00   3.45  1.95  0.60   
13  Khris Middleton   0.520   0.928   7.83  61.29  18.00  17.34  2.88  0.39   
14    Pascal Siakam   0.454   0.802   2.70  58.95  23.85  13.65  3.45  1.80   

      TOV  
0    5.28  
1    5.61  
2    3.00  
3    4.77  
4    4.80  
5   11.31  
6    3.81  
7    2.25  
8    5.85  
9    2.43  
10   5.43  
11   2.61  
12   1.80  
13   6.78  
14   7.20

In [2]:
TOTAL_ROW = "total"
def get_total_row(roster_dataframe):
    """
    Function creates a row in the provided parameter dataframe which contains the total for
    categories that should be summed and averages for categories that should be averaged
    param roster_dataframe (pandas dataframe): dataframe containing player week 9cat stats
    return roster_dataframe_with_total (pandas dataframe): dataframe containing total row
    """
    sum_9cat_column_dataframe = pd.DataFrame(roster_dataframe.iloc[:, 3:].sum()).T
    mean_9cat_column_dataframe =  pd.DataFrame(roster_dataframe.iloc[:, 1:3].mean()).T
    total_dataframe = pd.concat([mean_9cat_column_dataframe,  sum_9cat_column_dataframe], axis=1)
    total_dataframe[PLAYER_COLUMN] = TOTAL_ROW
    roster_dataframe_with_total = roster_dataframe.append(total_dataframe)
    roster_dataframe_with_total.reset_index(inplace=True, drop=True)
    return roster_dataframe_with_total

In [73]:
b = get_total_row(predicted_weekly_df)



In [80]:
af.format_dataframe_decimals(b)



ValueError: Unknown format code 'f' for object of type 'str'

In [3]:
def get_teams_weekly_predicted_stats_pipeline(team_dict, week="next", season=2020,
                                              team_9cat_average_stats_data=None):
    """
    Function pipelines the process required to find a league teams weekly predicted 9cat totals.
    Initially, the number of games for each player in the provided parameter week is found.
    Average 9cat player stats are multiplied by that number and the entire roster stats are
    summed to provide a total. The dataframe is finally formatted.
    param team_dict (dictionary): dictionary of team from league
    param week (string): determines which week to calculate weekly games for
    param season (int): determines which season to get NBA schedule for
    param team_9cat_average_stats_data (pandas dataframe): dataframe containing player 9cat season
    averages
    return roster_dataframe_with_total (pandas dataframe): dataframe containing player 9cat
    predicted weekly stats and entire roster totals
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season)
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    if team_9cat_average_stats_data == None:
        team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dict, league)
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_dataframe)
    else:
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_data)
    roster_dataframe_with_total = get_total_row(predicted_weekly_9cat_stats)
    roster_dataframe_with_total = af.format_dataframe_decimals(roster_dataframe_with_total)
    return roster_dataframe_with_total

In [79]:
get_teams_weekly_predicted_stats_pipeline(af.NUNN_OF_YALL_BETTA, week="current")

[2021-02-07 19:40:38,378 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 19:40:38,381 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 230.8955225944519
[2021-02-07 19:40:38,382 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-07 19:40:39,020 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 19:40:39,025 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 231.54037284851074
[2021-02-07 19:40:39,027 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-07 19:40:43,664 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 19:40:43,667 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 236.18284964561462
[2021-02-07 19:40:43,669 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-07 19:40:44,692 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-07 19:40:44,695 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 237.2106738090515
[2021-02-07 19:40:44,696 DEBUG] [yahoo_oauth.oauth

PLAYER    FG_PCT  FT_PCT   FG3M     PTS     REB     AST    STL  \
0      Marcus Smart  0.394000   0.742   4.95   39.18    8.28   18.18   5.46   
1   Malcolm Brogdon  0.450000   0.889   7.86   63.36   11.64   19.50   3.87   
2       Jeremy Lamb  0.511000   0.966   6.00   38.19   12.54    4.35   2.73   
3   Duncan Robinson  0.424000   0.861   9.78   37.83   10.95    4.95   1.17   
4      Jimmy Butler  0.452000   0.842   0.00   50.73   18.54   17.73   5.73   
5      Nikola Jokic  0.569000   0.840   4.23   81.54   34.35   25.77   5.04   
6       Enes Kanter  0.596000   0.765   0.00   34.23   32.73    3.27   1.77   
7     Davis Bertans  0.349000   0.806   7.59   30.72    7.95    3.18   1.59   
8      Eric Bledsoe  0.444000   0.651   6.87   41.13   11.01   11.13   2.01   
9   Cameron Johnson  0.443000   0.870   6.72   33.87   11.01    4.71   2.01   
10     Theo Maledon  0.397000   0.632   4.29   22.56    7.86    8.58   2.85   
11      Patty Mills  0.453000   0.938   7.71   40.83    5.73    8.10   1.95   
12   Gary Trent Jr.  0.421000   0.708   9.15   42.00    6.00    3.45   1.95   
13  Khris Middleton  0.520000   0.928   7.83   61.29   18.00   17.34   2.88   
14    Pascal Siakam  0.454000   0.802   2.70   58.95   23.85   13.65   3.45   
15            total  0.458467   0.816  85.68  676.41  220.44  163.89  44.46   

      BLK    TOV  
0    1.59   5.28  
1    0.75   5.49  
2    2.46   2.73  
3    0.90   4.56  
4    1.35   4.65  
5    2.04  11.31  
6    2.73   3.81  
7    0.87   2.13  
8    0.87   5.85  
9    0.99   2.43  
10   0.15   5.43  
11   0.12   2.61  
12   0.60   1.80  
13   0.39   6.78  
14   1.80   7.20  
15  17.61  72.06

### All Together


In [13]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        try:
            dataframe[column] = dataframe[column].astype(float)
        except ValueError:
            pass
    return dataframe

def get_and_format_team_9cat(team, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns and returns a color coded and decimal
    place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = dataframe.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """
    Function authenticates on Yahoo and creates creates two dataframes containing team roster
    9cat averages for each team specified in the parameters. It calculates the difference between
    both 9cat averages and finally it formats the values for visual ease.
    params team1 (dictionary): team dictionary
    params team2 (dictionary): team dictionary
    return matchup_difference (dataframe): team difference 9cat dataframe
    return colored_matchup_difference (dataframe): formatted team difference 9cat dataframe
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        player_games_dictionary[player] = \
            team_game_counts_dataframe[team_game_counts_dataframe[INDEX_COLUMN] == team][0].iloc[0]
    return player_games_dictionary

def get_predicted_player_weekly_9cat(player_games_dictionary, team_9cat_average_stats):
    """
    Function is used to create a dataframe which contains predicted totals for a week of Yahoo
    Fantasy play. For each player, 9cat features are multiplied by the amount of games that
    player plays in one week.
    param player_games_dictionary (dictionary): key/value pairing of player names/games playing
    param team_9cat_average_stats (pandas dataframe): dataframe containing the season average
    9cat stats for each player on a team
    return predicted_9cat_stats (pandas dataframe): dataframe with 9cat averages multiplied by
    games played in a week
    """
    predicted_9cat_stats = pd.DataFrame()
    team_9cat_average_stats = columns_to_dtype_float(team_9cat_average_stats)
    for player in player_games_dictionary.keys():
        player_predicted_9cat_stats = pd.concat(
            [team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, :3],
            (team_9cat_average_stats[team_9cat_average_stats[PLAYER_COLUMN] == player].iloc[:, 3:]
            * 3)], axis=1)
        predicted_9cat_stats = player_predicted_9cat_stats.append(predicted_9cat_stats)
    predicted_9cat_stats.sort_index(inplace=True)
    return predicted_9cat_stats

def get_total_row(roster_dataframe):
    """
    Function creates a row in the provided parameter dataframe which contains the total for
    categories that should be summed and averages for categories that should be averaged
    param roster_dataframe (pandas dataframe): dataframe containing player week 9cat stats
    return roster_dataframe_with_total (pandas dataframe): dataframe containing total row
    """
    sum_9cat_column_dataframe = pd.DataFrame(roster_dataframe.iloc[:, 3:].sum()).T
    mean_9cat_column_dataframe =  pd.DataFrame(roster_dataframe.iloc[:, 1:3].mean()).T
    total_dataframe = pd.concat([mean_9cat_column_dataframe,  sum_9cat_column_dataframe], axis=1)
    total_dataframe[PLAYER_COLUMN] = TOTAL_ROW
    roster_dataframe_with_total = roster_dataframe.append(total_dataframe)
    roster_dataframe_with_total.reset_index(inplace=True, drop=True)
    return roster_dataframe_with_total

def get_teams_weekly_predicted_stats_pipeline(team_dict, week="next", season=2020,
                                              team_9cat_average_stats_data=None):
    """
    Function pipelines the process required to find a league teams weekly predicted 9cat totals.
    Initially, the number of games for each player in the provided parameter week is found.
    Average 9cat player stats are multiplied by that number and the entire roster stats are
    summed to provide a total. The dataframe is finally formatted.
    param team_dict (dictionary): dictionary of team from league
    param week (string): determines which week to calculate weekly games for
    param season (int): determines which season to get NBA schedule for
    param team_9cat_average_stats_data (pandas dataframe): dataframe containing player 9cat season
    averages
    return roster_dataframe_with_total (pandas dataframe): dataframe containing player 9cat
    predicted weekly stats and entire roster totals
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season)
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    if team_9cat_average_stats_data == None:
        team_9cat_average_stats_dataframe = af.team_9cat_average_stats(team_dict, league)
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_dataframe)
    else:
        predicted_weekly_9cat_stats = get_predicted_player_weekly_9cat(player_games_dictionary,
                                                                 team_9cat_average_stats_data)
    roster_dataframe_with_total = get_total_row(predicted_weekly_9cat_stats)
    roster_dataframe_with_total = af.format_dataframe_decimals(roster_dataframe_with_total)
    return roster_dataframe_with_total

def get_filtered_total_row(total_dataframe, team_dict):
    """
    Function filters a dataframe containing weekly total 9cat stats and returns the *total* row
    so that it can be used for differencing. The function appends the team name to the total row
    param total_dataframe (pandas dataframe): dataframe containing weekly total 9cat stats
    param team_dict (dictionary): dictionary of team from league
    return total_row_dataframe (pandas dataframe): single row dataframe containing total 9cat
    stats with team name
    return total_row_dataframe_no_name (pandas dataframe): single row dataframe containing total 9cat
    stats without team name
    """
    total_row_dataframe = total_dataframe[total_dataframe[PLAYER_COLUMN] == TOTAL_ROW]
    total_row_dataframe[PLAYER_COLUMN] = team_dict[NAME_KEY]
    total_row_dataframe.index = [TOTAL_ROW]
    total_row_dataframe_no_name = total_row_dataframe.drop(PLAYER_COLUMN, axis=1)
    return total_row_dataframe, total_row_dataframe_no_name

def weekly_matchup_evaluator(team1_dict, team2_dict, team1_9cat_average_stats_data=None,
                             team2_9cat_average_stats_data=None, week="next", season=2020):
    """

    """
    team1_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team1_dict, week=week, season=season,
        team_9cat_average_stats_data=team1_9cat_average_stats_data)
    team2_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team2_dict, week=week, season=season,
        team_9cat_average_stats_data=team2_9cat_average_stats_data)
    team1_total_row_dataframe, team1_total_row_dataframe_no_name = get_filtered_total_row\
        (team1_roster_dataframe_with_total, team1_dict)
    team2_total_row_dataframe, team2_total_row_dataframe_no_name = get_filtered_total_row\
        (team2_roster_dataframe_with_total, team2_dict)
    team1_total_row_dataframe_no_name = columns_to_dtype_float(team1_total_row_dataframe_no_name)
    team2_total_row_dataframe_no_name = columns_to_dtype_float(team2_total_row_dataframe_no_name)
    total_difference_dataframe = team1_total_row_dataframe_no_name.subtract\
        (team2_total_row_dataframe_no_name)
    total_difference_dataframe_colored = applymap_color_and_format(total_difference_dataframe)
    return total_difference_dataframe, total_difference_dataframe_colored

In [24]:
nunn_of_yall_betta_weekly_stats = get_teams_weekly_predicted_stats_pipeline(af.NUNN_OF_YALL_BETTA,
                                                               week="current")
nunn_of_yall_betta_weekly_stats

[2021-02-08 20:34:41,216 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 20:34:41,217 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 6341.85152554512
[2021-02-08 20:34:41,218 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-02-08 20:34:41,219 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2021-02-08 20:34:43,581 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 20:34:43,590 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2.3692824840545654
[2021-02-08 20:34:43,591 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-08 20:34:49,194 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 20:34:49,196 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 7.976183652877808
[2021-02-08 20:34:49,197 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-08 20:34:51,685 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 20:34:51,688 DEBUG] [yahoo_oauth.oauth.token_is_valid

PLAYER FG_PCT FT_PCT   FG3M     PTS     REB     AST    STL  \
0      Theo Maledon  0.397  0.632   4.29   22.56    7.86    8.58   2.85   
1   Malcolm Brogdon  0.447  0.890   7.86   65.01   12.00   20.01   3.99   
2       Patty Mills  0.453  0.938   7.71   40.83    5.73    8.10   1.95   
3   Khris Middleton  0.520  0.928   7.83   61.29   18.00   17.34   2.88   
4      Jimmy Butler  0.444  0.860   0.00   54.54   20.46   19.65   6.00   
5      Nikola Jokic  0.569  0.840   4.23   81.54   34.35   25.77   5.04   
6     Davis Bertans  0.340  0.806   7.59   30.72    7.95    3.18   1.77   
7     Pascal Siakam  0.454  0.802   2.70   58.95   23.85   13.65   3.45   
8      Eric Bledsoe  0.444  0.651   6.87   41.13   11.01   11.13   2.01   
9   Cameron Johnson  0.446  0.885   6.96   34.65   11.19    4.65   1.92   
10     Marcus Smart  0.394  0.742   4.95   39.18    8.28   18.18   5.46   
11      Jeremy Lamb  0.505  0.966   6.00   38.73   12.54    4.65   3.00   
12  Duncan Robinson  0.426  0.838  10.17   39.00   11.34    5.10   1.17   
13      Enes Kanter  0.596  0.765   0.00   34.23   32.73    3.27   1.77   
14   Gary Trent Jr.  0.421  0.708   9.15   42.00    6.00    3.45   1.95   
15            total  0.457  0.817  86.31  684.36  223.29  166.71  45.21   

      BLK    TOV  
0    0.15   5.43  
1    0.75   5.88  
2    0.12   2.61  
3    0.39   6.78  
4    1.35   5.46  
5    2.04  11.31  
6    0.87   2.13  
7    1.80   7.20  
8    0.87   5.85  
9    0.96   2.46  
10   1.59   5.28  
11   2.46   2.73  
12   0.90   4.71  
13   2.73   3.81  
14   0.60   1.80  
15  17.58  73.44

In [2]:
magic_johnson_stats = get_teams_weekly_predicted_stats_pipeline(af.MAGICS_JOHNSON,
                                                               week="current")
magic_johnson_stats

[2021-02-08 19:36:18,827 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:36:18,829 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2839.4636821746826
[2021-02-08 19:36:18,830 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-08 19:36:19,567 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:36:19,569 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2840.2031564712524
[2021-02-08 19:36:19,570 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-08 19:36:24,491 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:36:24,495 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2845.12895154953
[2021-02-08 19:36:24,495 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-08 19:36:25,516 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-08 19:36:25,519 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2846.1532299518585
[2021-02-08 19:36:25,519 DEBUG] [yahoo_oauth.oauth

PLAYER FG_PCT FT_PCT   FG3M     PTS     REB     AST    STL  \
0       Fred VanVleet  0.409  0.887   9.90   60.66   13.17   18.90   5.34   
1       Mikal Bridges  0.483  0.830   6.00   42.27   16.77    6.42   1.77   
2       Kawhi Leonard  0.504  0.891   5.43   78.00   17.28   15.30   5.28   
3       Luguentz Dort  0.421  0.767   6.00   35.01    9.87    3.99   2.58   
4       Julius Randle  0.477  0.801   4.92   68.16   33.00   17.88   2.28   
5        LeBron James  0.497  0.712   8.25   76.38   23.13   23.37   3.00   
6         Cody Zeller  0.467  0.741   0.27   28.65   24.54    7.92   2.46   
7       Malik Beasley  0.453  0.857   9.00   60.39   15.00    7.05   2.22   
8   Tyrese Haliburton  0.487  0.867   6.87   35.58   11.01   16.29   3.57   
9    Bojan Bogdanovic  0.422  0.855   8.64   47.76   11.49    6.36   0.99   
10         Alec Burks  0.417  0.805   6.93   39.93   13.38    6.93   3.00   
11         Lonzo Ball  0.422  0.625   8.04   40.26   12.96   13.89   4.11   
12      Kevin Huerter  0.431  0.789   6.90   36.78   10.95   10.83   3.51   
13     Nikola Vucevic  0.482  0.846   7.86   69.36   34.14   10.26   3.12   
14              total  0.455  0.805  95.01  719.19  246.69  165.39  43.23   

      BLK    TOV  
0    2.22   5.34  
1    3.27   2.19  
2    1.86   5.28  
3    0.87   3.72  
4    0.60   9.84  
5    1.38  10.86  
6    2.19   1.92  
7    0.51   4.56  
8    1.86   4.56  
9    0.12   4.26  
10   1.14   3.69  
11   1.74   6.33  
12   0.51   3.78  
13   1.62   5.01  
14  19.89  71.34

In [26]:
nyb_total_row_dataframe, nyb_total_row_dataframe_no_name = get_filtered_total_row\
    (nunn_of_yall_betta_weekly_stats, af.NUNN_OF_YALL_BETTA)

mjs_total_row_dataframe, mjs_total_row_dataframe_no_name = get_filtered_total_row\
    (magic_johnson_stats, af.MAGICS_JOHNSON)


In [30]:
nyb_total_row_dataframe_no_name = columns_to_dtype_float(nyb_total_row_dataframe_no_name)
mjs_total_row_dataframe_no_name = columns_to_dtype_float(mjs_total_row_dataframe_no_name)

total_difference_dataframe = nyb_total_row_dataframe_no_name.subtract\
    (mjs_total_row_dataframe_no_name)


In [32]:
applymap_color_and_format(total_difference_dataframe)

In [22]:
def weekly_matchup_evaluator(team1_dict, team2_dict, team1_9cat_average_stats_data=None,
                             team2_9cat_average_stats_data=None, week="next", season=2020):
    """

    """
    team1_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team1_dict, week=week, season=season,
        team_9cat_average_stats_data=team1_9cat_average_stats_data)
    team2_roster_dataframe_with_total = get_teams_weekly_predicted_stats_pipeline(
        team2_dict, week=week, season=season,
        team_9cat_average_stats_data=team2_9cat_average_stats_data)
    team1_total_row_dataframe, team1_total_row_dataframe_no_name = get_filtered_total_row\
        (team1_roster_dataframe_with_total, team1_dict)
    team2_total_row_dataframe, team2_total_row_dataframe_no_name = get_filtered_total_row\
        (team2_roster_dataframe_with_total, team2_dict)
    team1_total_row_dataframe_no_name = columns_to_dtype_float(team1_total_row_dataframe_no_name)
    team2_total_row_dataframe_no_name = columns_to_dtype_float(team2_total_row_dataframe_no_name)
    total_difference_dataframe = team1_total_row_dataframe_no_name.subtract\
        (team2_total_row_dataframe_no_name)
    total_difference_dataframe_colored = applymap_color_and_format(total_difference_dataframe)
    return total_difference_dataframe, total_difference_dataframe_colored

In [25]:
total_difference_dataframe, total_difference_dataframe_colored = weekly_matchup_evaluator(
    af.NUNN_OF_YALL_BETTA, af.MAGICS_JOHNSON, team1_9cat_average_stats_data=None,
    team2_9cat_average_stats_data=None, week="current", season=2020)

[2021-02-10 20:47:36,710 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-10 20:47:36,712 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1373.2326700687408
[2021-02-10 20:47:36,713 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-10 20:47:37,408 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-10 20:47:37,411 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1373.9320256710052
[2021-02-10 20:47:37,412 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-10 20:47:41,360 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-10 20:47:41,364 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1377.8846974372864
[2021-02-10 20:47:41,365 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-10 20:47:42,333 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-10 20:47:42,337 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1378.857450723648
[2021-02-10 20:47:42,337 DEBUG] [yahoo_oauth.oaut

In [26]:
total_difference_dataframe

FG_PCT  FT_PCT  FG3M   PTS    REB   AST   STL   BLK   TOV
total  -0.003   0.008 -6.84 -27.9 -25.56  1.41  2.22 -4.02  1.59

## Current Week Matchup Stats
This section will use the Yahoo Fantasy API to check on the current matchup

In [225]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

[2021-02-13 00:08:04,006 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-13 00:08:04,007 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 4221.378016233444
[2021-02-13 00:08:04,009 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-02-13 00:08:04,009 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [226]:
matchups_w8 = league.matchups(week=8)

In [295]:
FANTASY_CONTENT_KEY = "fantasy_content"
LEAGUE_KEY = "league"
SCOREBOARD_KEY = "scoreboard"
ZERO_KEY = "0"
MATCHUPS_KEY = "matchups"
MATCHUP_KEY = "matchup"
TEAMS_KEY = "teams"
ONE_KEY = "1"
TEAM_KEY = "team"
NAME_KEY = "name"

def get_teams_matchup_number(team_dict, matchups):
    """
    Function loops through returned the returned Yahoo Fantasy matchup nested dictionary to find
    the matchup number of the specified parameter name
    """
    for matchup in range(len(matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY]) - 1):
        if matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ZERO_KEY]\
                [TEAM_KEY][0][2][NAME_KEY] == team_dict[NAME_KEY] \
                or matchups[FANTASY_CONTENT_KEY][LEAGUE_KEY][1][SCOREBOARD_KEY][ZERO_KEY]\
                [MATCHUPS_KEY][str(matchup)][MATCHUP_KEY][ZERO_KEY][TEAMS_KEY][ONE_KEY]\
                [TEAM_KEY][0][2][NAME_KEY] == team_dict[NAME_KEY]:
            return str(matchup)

In [297]:
get_teams_matchup_number(af.NUNN_OF_YALL_BETTA, matchups_w8)

# With matchup number create stats datafrmae for both teams in matchup

'3'

In [131]:
first_team_name = matchups_w8["fantasy_content"]["league"][1]["scoreboard"]["0"]["matchups"]["1"]\
    ["matchup"]["0"]["teams"]["1"]["team"][0][2]["name"]
stat_8 = matchups_w8["fantasy_content"]["league"][1]["scoreboard"]["0"]["matchups"]["1"]\
    ["matchup"]["0"]["teams"]["1"]["team"][1]["team_stats"]["stats"][3]["stat"]["value"]

In [130]:
pd.DataFrame.from_dict(stat_8, orient='index', columns=["a"])

AttributeError: 'str' object has no attribute 'values'

In [144]:
current_matchup_stats = matchups_w8["fantasy_content"]["league"][1]["scoreboard"]["0"]["matchups" \
                                                                                    ""]["1"]\
    ["matchup"]["0"]["teams"]["1"]["team"][1]["team_stats"]["stats"]
current_matchup_stats[5]

{'stat': {'stat_id': '12', 'value': '277'}}

In [158]:
current_9cat_stats_list = list()
for index in range(len(current_matchup_stats)):
    if index != 0 and index != 2:
        current_9cat_stats_list.append(current_matchup_stats[index]["stat"]["value"])

In [159]:
current_9cat_stats_list


['.444', '.795', '23', '277', '100', '71', '14', '8', '31']

In [172]:
COLUMNS = [af.PLAYER_COLUMN] + af.NINE_CAT_COLUMNS
VALUES = [first_team_name] + current_9cat_stats_list
live_matchup_9cat_stats = pd.DataFrame(VALUES).T
live_matchup_9cat_stats.columns = COLUMNS
live_matchup_9cat_stats

PLAYER FG_PCT FT_PCT FG3M  PTS  REB AST STL BLK TOV
0  Wakanda Forever   .444   .795   23  277  100  71  14   8  31